# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [5]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\edelb\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
# Read your api key environment variable
load_dotenv("../Starter_Code/.env")
api_key = os.getenv("news_api")
print(api_key)

9a2d0207baf44931bde4e6ab90344986


In [7]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=api_key)

In [8]:
# Fetch the Bitcoin news articles
btc_news = newsapi.get_everything(
    q="bitcoin",
    language="en",
    sort_by="relevancy"
)

In [9]:
# Fetch the Ethereum news articles
eth_news = newsapi.get_everything(
    q="ethereum",
    language="en",
    sort_by="relevancy"
)

In [10]:
# Create the Bitcoin sentiment scores DataFrame
btc_sentiments = []

for article in btc_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(btc_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()


,date,text,compound,positive,negative,neutral
0,2022-05-10,"Now, even though there are a number of women-f...",0.0772,0.036,0.000,0.964
1,2022-05-04,A Bitcoin mining site powered by otherwise los...,-0.0516,0.056,0.061,0.882
2,2022-05-02,Warren Buffett has always been a bitcoin skept...,-0.3269,0.085,0.143,0.772
3,2022-05-16,"As a kid, I remember when my father tried to u...",0.3818,0.114,0.052,0.833
4,2022-05-09,"Image source, Getty Images\r\nThe value of Bit...",0.3400,0.072,0.000,0.928


In [11]:
# Create the Ethereum sentiment scores DataFrame
eth_sentiments = []

for article in eth_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
eth_df = pd.DataFrame(eth_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
eth_df = eth_df[cols]

eth_df.head()

,date,text,compound,positive,negative,neutral
0,2022-05-10,Meta has revealed more of how NFTs will work o...,0.6486,0.135,0.000,0.865
1,2022-05-23,GameStop has officially thrown itself headlong...,-0.1027,0.000,0.040,0.960
2,2022-05-02,A multi-billion dollar cryptocurrency company ...,-0.2263,0.046,0.075,0.879
3,2022-05-04,When Bored Ape Yacht Club creators Yuga Labs a...,-0.2732,0.000,0.055,0.945
4,2022-05-23,GameStop is going all-in on crypto. The video ...,0.1280,0.046,0.000,0.954


In [12]:
# Describe the Bitcoin Sentiment
btc_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,-0.119220,0.063350,0.084900,0.851800
std,0.404397,0.060985,0.075734,0.100743
min,-0.859300,0.000000,0.000000,0.557000
25%,-0.386825,0.000000,0.057250,0.827000
50%,-0.284600,0.055000,0.071500,0.878000
75%,0.156025,0.085000,0.103750,0.923000
max,0.750600,0.202000,0.300000,0.964000


In [13]:
# Describe the Ethereum Sentiment
eth_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.000000
mean,-0.088800,0.032800,0.048600,0.918700
std,0.343530,0.051042,0.043472,0.051321
min,-0.690800,0.000000,0.000000,0.822000
25%,-0.284450,0.000000,0.000000,0.875500
50%,-0.153100,0.000000,0.050000,0.935000
75%,0.045025,0.051750,0.069000,0.955000
max,0.690800,0.178000,0.178000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin 

Q: Which coin had the highest compound score?

A: Bitcoin

Q. Which coin had the highest positive score?

A: Bitcoin

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [14]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [15]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

# Expand the default stopwords list if necessary
sw_addon = {'a', 'of', 'to','the','it','in'}
sw = sw.union(sw_addon)

In [16]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a tokenized list of the words
    words = word_tokenize(text) 
    
    # Remove the punctuation from text
    words = list(filter(lambda t: t not in punctuation, words))    
    
    # Lemmatize words into root words
    tokens = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase
    words = list(filter(lambda w: w.lower(), words))    
    
    # Remove the stop words
    words = list(filter(lambda t: t.lower() not in sw, words))
    
    return tokens

In [17]:
# Create a new tokens column for Bitcoin
btc_df["tokens"] = btc_df.text.apply(tokenizer)
btc_df.head()

,date,text,compound,positive,negative,neutral,tokens
0,2022-05-10,"Now, even though there are a number of women-f...",0.0772,0.036,0.000,0.964,"[Now, even, though, there, are, a, number, of,..."
1,2022-05-04,A Bitcoin mining site powered by otherwise los...,-0.0516,0.056,0.061,0.882,"[A, Bitcoin, mining, site, powered, by, otherw..."
2,2022-05-02,Warren Buffett has always been a bitcoin skept...,-0.3269,0.085,0.143,0.772,"[Warren, Buffett, ha, always, been, a, bitcoin..."
3,2022-05-16,"As a kid, I remember when my father tried to u...",0.3818,0.114,0.052,0.833,"[As, a, kid, I, remember, when, my, father, tr..."
4,2022-05-09,"Image source, Getty Images\r\nThe value of Bit...",0.3400,0.072,0.000,0.928,"[Image, source, Getty, Images, The, value, of,..."


In [18]:
# Create a new tokens column for Ethereum
eth_df["tokens"] = eth_df.text.apply(tokenizer)
eth_df.head()

,date,text,compound,positive,negative,neutral,tokens
0,2022-05-10,Meta has revealed more of how NFTs will work o...,0.6486,0.135,0.000,0.865,"[Meta, ha, revealed, more, of, how, NFTs, will..."
1,2022-05-23,GameStop has officially thrown itself headlong...,-0.1027,0.000,0.040,0.960,"[GameStop, ha, officially, thrown, itself, hea..."
2,2022-05-02,A multi-billion dollar cryptocurrency company ...,-0.2263,0.046,0.075,0.879,"[A, multi-billion, dollar, cryptocurrency, com..."
3,2022-05-04,When Bored Ape Yacht Club creators Yuga Labs a...,-0.2732,0.000,0.055,0.945,"[When, Bored, Ape, Yacht, Club, creator, Yuga,..."
4,2022-05-23,GameStop is going all-in on crypto. The video ...,0.1280,0.046,0.000,0.954,"[GameStop, is, going, all-in, on, crypto, The,..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [19]:
from collections import Counter
from nltk import ngrams

In [20]:
# Generate the Bitcoin N-grams where N=2
N = 2
btc_grams = ngrams(tokenizer(btc_df.text.str.cat()), N)
Counter(btc_grams).most_common(10)

[(('a', 'new'), 6),
 (('for', 'the'), 5),
 (('the', 'world'), 5),
 (('world', "'s"), 4),
 (('Reuters', 'Bitcoin'), 4),
 (('char', 'May'), 4),
 (('in', 'the'), 4),
 (('the', 'only'), 3),
 (('it', 'previous'), 3),
 (('previous', 'close'), 3)]

In [21]:
# Generate the Ethereum N-grams where N=2
N = 2
eth_grams = ngrams(tokenizer(eth_df.text.str.cat()), N)
Counter(eth_grams).most_common(10)

[(('the', 'world'), 4),
 (('world', "'s"), 4),
 (('char', 'May'), 3),
 (('Reuters', 'Bitcoin'), 3),
 (('it', 'previous'), 3),
 (('previous', 'close'), 3),
 (('close', 'Bitcoin'), 3),
 (('Bitcoin', 'the'), 3),
 (("'s", 'biggest'), 3),
 (('biggest', 'and'), 3)]

In [22]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [23]:
# Use token_count to get the top 10 words for Bitcoin
btc_tokens = tokenizer(btc_df.text.str.cat())
token_count(btc_tokens, 10)

[('the', 31),
 ('a', 21),
 ('to', 21),
 ('char', 19),
 ('of', 13),
 ('in', 11),
 ('Bitcoin', 10),
 ('it', 10),
 ("'s", 10),
 ('cryptocurrency', 9)]

In [24]:
# Use token_count to get the top 10 words for Ethereum
eth_tokens = tokenizer(eth_df.text.str.cat())
token_count(eth_tokens, 10)

[('to', 26),
 ('a', 24),
 ('the', 23),
 ('char', 20),
 ('it', 16),
 ('cryptocurrency', 12),
 ('of', 11),
 ('The', 10),
 ('on', 9),
 ('Bitcoin', 9)]

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [25]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [26]:
# Generate the Bitcoin word cloud
btc_wordcloud = WordCloud(colormap="RdYlBu").generate(btc_tokens)
plt.imshow(wordcloud)
plt.axis("off")
fontdict = {"fontsize": 20, "fontweight": "bold"}
plt.title("Money News Word Cloud", fontdict=fontdict)
plt.show()


TypeError: expected string or bytes-like object

In [27]:
# Generate the Ethereum word cloud
eth_wordcloud = WordCloud(colormap="RdYlBu").generate(eth_tokens)
plt.imshow(wordcloud)
plt.axis("off")
fontdict = {"fontsize": 20, "fontweight": "bold"}
plt.title("Money News Word Cloud", fontdict=fontdict)
plt.show()


TypeError: expected string or bytes-like object

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [28]:
import spacy
from spacy import displacy

ModuleNotFoundError: No module named 'spacy'

In [25]:
# Download the language model for SpaCy
 !python -m spacy download en_core_web_sm

In [26]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [29]:
# Concatenate all of the Bitcoin text together
btc_txt = btc_df.text.str.cat()

In [31]:
# Run the NER processor on all of the text
doc = nlp(btc_txt)

# Add a title to the document
doc.user_data["title"] = "BTC NER"

NameError: name 'nlp' is not defined

In [29]:
# Render the visualization
displacy.render(doc, style='ent')

In [30]:
# List all Entities
for entity in doc.ents:
    print(entity.text, entity.label_)

---

### Ethereum NER

In [31]:
# Concatenate all of the Ethereum text together
eth_txt = eth_df.text.str.cat()

In [32]:
# Run the NER processor on all of the text
doc1 = nlp(eth_txt)

# Add a title to the document
doc.user_data["title"] = "ETH NER"

In [33]:
# Render the visualization
displacy.render(doc1, style='ent')

In [32]:
# List all Entities
for entity1 in doc.ents:
    print(entity1.text, entity1.label_)

NameError: name 'doc' is not defined

---